Load libraries.  In this case we need the ndex library, which you can get with "pip install ndex"

In [1]:
#ndex = __import__("ndex")
import ndex2.client as nc2
import ndex2

In [2]:
import requests
import json
import base64

# Connect to NDEx in order to retrieve a network.

In [3]:
anon_ndex = nc2.Ndex2("http://public.ndexbio.org")

Here we grab the "NCI Pathway Interaction Database - Final Revision - Extended Binary SIF" from NDEx, using its UUID.  The NdexGraph object is an extension of the networkx objects, see https://networkx.github.io/ for all the things you can do with it

In [4]:
G = ndex2.create_nice_cx_from_server(server='http://public.ndexbio.org', uuid='c0e70804-d848-11e6-86b1-0ac135e8bacf')

In [5]:
nodeid = list(G.nodes.keys())[0]
G.remove_node_attribute(node=nodeid, attribute_name='diffusion_input')
G.set_node_attribute(type='double', node=nodeid, attribute_name='diffusion_input', values='1.0')
print(G.get_node_attribute_value(node=nodeid, attribute_name='diffusion_input'))

1.0


Call the Diffusion service

#### NOTE: Diffusion uses the CXmate service, which requires all attributes to be passed as strings. So we convert the network to CX and stringify attributes

In [21]:
url = 'http://v3.heat-diffusion.cytoscape.io'
payload = G.to_cx()
for p in payload:
    k = list(p.keys())[0]
    if 'Attributes' in k:
        for i in range(len(p[k])):
            p[k][i]['v'] = str(p[k][i]['v'])
response = requests.post(url, json=payload)

#The 'data' dictionary in the json response is a dictionary with node IDs as keys and, 'rank' and 'heat' as the values. 

See the top 10 most relevant nodes to your query. 

In [20]:
R_nice = ndex2.create_nice_cx_from_raw_cx(response.json()['data'])
R = R_nice.to_networkx()

nos = []
for n in R.nodes_iter():
    if int(R.node[n]['diffusion_output_rank']) < 10:
        R.node[n]['nid'] = n
        nos.append(R.node[n])
nos = sorted(nos, key=lambda k: k['diffusion_output_rank'])
for no in nos:
    print("id: " + str(no['nid']) + " name: " + no['name'] + \
          " rank: " + str(no['diffusion_output_rank']) + \
          " heat: " + str(no['diffusion_output_heat']))


networkx version 1.11
id: 3431 name: CMTM2 rank: 0 heat: 0.905968155561
id: 2878 name: BRAF rank: 1 heat: 0.0631120532008
id: 2600 name: PAK4 rank: 2 heat: 0.0304622579003
id: 2147 name: PPP2R2B rank: 3 heat: 0.0303269187099
id: 2397 name: KSR1 rank: 4 heat: 0.0250882472905
id: 1337 name: RIT1 rank: 5 heat: 0.0221534372756
id: 1340 name: RIT2 rank: 6 heat: 0.0221534372756
id: 1857 name: PPP2R1A rank: 7 heat: 0.0203617273203
id: 1251 name: RANBP10 rank: 8 heat: 0.0198381840218
id: 1431 name: MUC20 rank: 9 heat: 0.0188216924195
